In [9]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        """추상클래스"""

class FidelityPromo(Promotion):

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 100 else 0
    
class BulkItemPromo(Promotion):
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

class LargeOrderPromo(Promotion):
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total()*.07
        return 0


In [10]:
joe = Customer('John Deo', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [11]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [12]:
banana_cart = [LineItem('banana', 30, .5),
                LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [13]:
long_order = [LineItem(str(item_code),1,1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [14]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [22]:
'''이전 예시 리팩토링'''
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

def fidelity_promo(order):
    return order.total() * .05 if order.customer.fidelity >= 100 else 0
    
def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total()*.07
    return 0


In [23]:
joe = Customer('John Deo', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
Order(joe, cart, fidelity_promo)


<Order total: 42.00 due: 42.00>

In [24]:
Order(ann, cart, fidelity_promo)


<Order total: 42.00 due: 39.90>

In [25]:
banana_cart = [LineItem('banana', 30, .5),
                LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, bulk_item_promo)


<Order total: 30.00 due: 28.50>

In [26]:
long_order = [LineItem(str(item_code),1,1.0) for item_code in range(10)]
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [27]:
Order(joe, cart, large_order_promo())


TypeError: large_order_promo() missing 1 required positional argument: 'order'

In [29]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]
def best_promo(order):
    return max(promo(order) for promo in promos)
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [30]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [31]:
Order(joe, cart, best_promo)

<Order total: 42.00 due: 42.00>

In [32]:
promos = [globals()[name]for name in globals()
            if name.endswith('_promo')
            and name != 'best_promo']

def best_promo(order):
    return max(promo(order) for promo in promos)

'''아래와 같은 일을 수행
promos = [fidelity_promo, bulk_item_promo, large_order_promo]
def best_promo(order):
    return max(promo(order) for promo in promos)
Order(joe, long_order, best_promo)2re
'''

'아래와 같은 일을 수행\npromos = [fidelity_promo, bulk_item_promo, large_order_promo]\ndef best_promo(order):\n    return max(promo(order) for promo in promos)\nOrder(joe, long_order, best_promo)\n'

In [1]:
import inspect
import promotions
promos = [func for name, func in inspect.getmembers(promotions, inspect.isfuntion)]

TypeError: 'ABCMeta' object is not subscriptable